# House Prices - Advanced Regression Techniques

## Predict sales prices and practice feature engineering, RFs, and gradient boosting

## 1. 라이브러리 선언

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
!pip install lightgbm
from lightgbm import LGBMRegressor
!pip install bayesian-optimization
from bayes_opt import BayesianOptimization
import numpy as np

## 2. EDA & 전처리

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
print('train_shape:', train.shape)
print('test_shape:', test.shape)

train_shape: (1460, 81)
test_shape: (1459, 80)


## 2-1. 결측치 확인

In [6]:
train.isnull().sum()

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

## 2-2. 결측치의 양이 40%가 넘는 칼럼 제거

In [7]:
num_null = train.isnull().sum() / len(train)
remove_col = num_null[num_null >= 0.4]
print(remove_col)

Alley          0.937671
FireplaceQu    0.472603
PoolQC         0.995205
Fence          0.807534
MiscFeature    0.963014
dtype: float64


In [8]:
train = train.drop(remove_col.keys(), axis = 1)
test = test.drop(remove_col.keys(), axis = 1)
print('train_shape:', train.shape)
print('test_shape:', test.shape)

train_shape: (1460, 76)
test_shape: (1459, 75)


In [9]:
X = train.drop(['SalePrice'], axis = 1)
y = train['SalePrice']
print('X_shape:', X.shape)
print('y_shape:', y.shape)

X_shape: (1460, 75)
y_shape: (1460,)


## 2-3. X 데이터와 test 데이터를 합친 뒤 칼럼을 문자와 숫자로 나누고 문자인 칼럼만 One-Hot Encoder 사용

In [10]:
boston = pd.concat((X, test), axis = 0)
print('boston_shape:', boston.shape)

boston_shape: (2919, 75)


In [11]:
boston_obj = boston.select_dtypes(include = 'object')
boston_num = boston.select_dtypes(exclude = 'object')
print('object_col:', boston_obj.columns)
print('num_col:', boston_num.columns)
print('object_shape:', boston_obj.shape)
print('num_shape:', boston_num.shape)

object_col: Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
       'PavedDrive', 'SaleType', 'SaleCondition'],
      dtype='object')
num_col: Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fire

In [12]:
boston_dum = pd.get_dummies(boston_obj,drop_first = True)
print('boston_dum_shape:', boston_dum.shape)

boston_dum_shape: (2919, 196)


In [13]:
boston = pd.concat([boston_dum,boston_num], axis = 1)
print('boston_shape:', boston.shape)
print('boston_len:', len(boston))

boston_shape: (2919, 233)
boston_len: 2919


In [14]:
train_one = boston[:len(X)]
test_one = boston[len(X):]
print('train_one_shape:', train_one.shape)
print('test_one_shape:', test_one.shape)
print('train_one_head:', train_one.head())
print('test_one_head:', test_one.head())

train_one_shape: (1460, 233)
test_one_shape: (1459, 233)
train_one_head:    MSZoning_FV  MSZoning_RH  MSZoning_RL  MSZoning_RM  Street_Pave  \
0            0            0            1            0            1   
1            0            0            1            0            1   
2            0            0            1            0            1   
3            0            0            1            0            1   
4            0            0            1            0            1   

   LotShape_IR2  LotShape_IR3  LotShape_Reg  LandContour_HLS  LandContour_Low  \
0             0             0             1                0                0   
1             0             0             1                0                0   
2             0             0             0                0                0   
3             0             0             0                0                0   
4             0             0             0                0                0   

   ...  GarageArea 

In [15]:
X = train_one

## 3. LGBMRegressor의 하이퍼파라미터를 BayesianOptimization으로 결정 

### 3-1. 하이퍼파라미터 범위

In [16]:
lgbm_parameter_bounds = {
    'learning_rate':(0.0001,0.01),
    'n_estimators' : (10,1000),
    'max_depth' : (1,10),
    'subsample' : (0.1,1),
    'colsample_bytree' : (0.5,1),
    'num_leaves': (24, 45),
    }

In [17]:
def lgbm_bo(learning_rate, n_estimators, max_depth, subsample, colsample_bytree, num_leaves):
    lgbm_params = {
        'learning_rate' : float(learning_rate),
        'n_estimators' : int(n_estimators),
        'max_depth' : int(max_depth),
        'subsample' : float(subsample),
        'colsample_bytree' : float(colsample_bytree),
        'num_leaves' : int(num_leaves),
    }
    lgbm = LGBMRegressor(**lgbm_params)
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2)
    lgbm.fit(X_train, y_train)
    mse_score = cross_val_score(lgbm, X_valid, y_valid, scoring = 'neg_mean_squared_error', cv = 5)
    score = np.mean(mse_score)
    return score

### 3-2. 점수가 제일 높은 하이퍼파라미터로 LGBMRegressor 적용

In [19]:
BO_lgbm = BayesianOptimization(f = lgbm_bo, pbounds = lgbm_parameter_bounds, random_state = 0)

#학습 시작
BO_lgbm.maximize(init_points = 10, n_iter = 100)

max_params = BO_lgbm.max['params']
max_params['learning_rate'] = float(max_params['learning_rate'])
max_params['n_estimators'] = int(max_params['n_estimators'])
max_params['max_depth'] = int(max_params['max_depth'])
max_params['subsample'] = float(max_params['subsample'])
max_params['colsample_bytree'] = float(max_params['colsample_bytree'])
max_params['num_leaves'] = int(max_params['num_leaves'])
print(max_params)

|   iter    |  target   | colsam... | learni... | max_depth | n_esti... | num_le... | subsample |
-------------------------------------------------------------------------------------------------
|  1        | -1.834e+0 |  0.7744   |  0.00718  |  6.425    |  549.4    |  32.9     |  0.6813   |
|  2        | -1.157e+0 |  0.7188   |  0.008929 |  9.673    |  389.6    |  40.63    |  0.576    |
|  3        | -2.314e+0 |  0.784    |  0.009263 |  1.639    |  96.26    |  24.42    |  0.8494   |
|  4        | -9.565e+0 |  0.8891   |  0.008713 |  9.808    |  801.2    |  33.69    |  0.8025   |
|  5        | -1.523e+0 |  0.5591   |  0.006435 |  2.29     |  945.2    |  34.96    |  0.4732   |
|  6        | -1.037e+0 |  0.6323   |  0.007765 |  5.105    |  572.7    |  24.39    |  0.6559   |
|  7        | -8.279e+0 |  0.806    |  0.006208 |  9.494    |  685.0    |  31.55    |  0.4933   |
|  8        | -3.6e+09  |  0.8488   |  0.000696 |  7.001    |  673.9    |  28.42    |  0.216    |
|  9        | -2.029

|  52       | -1.676e+0 |  0.9508   |  0.00595  |  5.721    |  484.8    |  40.94    |  0.9447   |
|  53       | -8.499e+0 |  0.5435   |  0.00674  |  4.441    |  823.1    |  41.28    |  0.8678   |
|  54       | -1.125e+0 |  0.5314   |  0.005631 |  2.112    |  780.3    |  41.31    |  0.6751   |
|  55       | -3.068e+0 |  0.7347   |  0.001013 |  4.977    |  957.9    |  35.04    |  0.5515   |
|  56       | -1.641e+0 |  0.6326   |  0.003608 |  6.579    |  484.9    |  41.89    |  0.7965   |
|  57       | -8.125e+0 |  0.7255   |  0.007666 |  9.251    |  622.1    |  34.02    |  0.3018   |
|  58       | -8.925e+0 |  0.6505   |  0.007301 |  9.068    |  320.7    |  38.99    |  0.3562   |
|  59       | -1.988e+0 |  0.9102   |  0.005017 |  1.225    |  564.4    |  30.22    |  0.7991   |
|  60       | -1.292e+0 |  0.7459   |  0.008653 |  5.018    |  592.1    |  36.97    |  0.7226   |
|  61       | -1.295e+0 |  0.8894   |  0.003998 |  8.294    |  545.5    |  40.35    |  0.3223   |
|  62       | -5.552

|  103      | -4.39e+09 |  0.5499   |  0.001641 |  4.859    |  174.0    |  33.11    |  0.3893   |
|  104      | -2.235e+0 |  0.7266   |  0.002696 |  1.922    |  438.7    |  42.81    |  0.4977   |
|  105      | -1.604e+0 |  0.749    |  0.008429 |  6.054    |  350.9    |  32.64    |  0.2426   |
|  106      | -1.023e+0 |  0.6241   |  0.008933 |  9.55     |  708.2    |  26.94    |  0.4288   |
|  107      | -4.376e+0 |  0.6074   |  0.000441 |  2.408    |  864.3    |  43.27    |  0.198    |
|  108      | -2.567e+0 |  0.6143   |  0.004972 |  9.895    |  222.0    |  27.85    |  0.569    |
|  109      | -2.094e+0 |  0.522    |  0.007014 |  9.467    |  158.5    |  26.98    |  0.6041   |
|  110      | -3.948e+0 |  0.8509   |  0.001854 |  7.688    |  526.5    |  29.94    |  0.394    |
{'colsample_bytree': 0.8238472358016229, 'learning_rate': 0.008952243102064422, 'max_depth': 9, 'n_estimators': 504, 'num_leaves': 31, 'subsample': 0.7749632785399652}


In [22]:
lgbm_tune = LGBMRegressor(learning_rate = 0.008952243102064422, max_depth = 9, n_estimators = 504, num_leaves = 31, subsample = 0.7749632785399652, colsample_bytree = 0.8238472358016229)
lgbm_tune.fit(X, y)

LGBMRegressor(colsample_bytree=0.8238472358016229,
              learning_rate=0.008952243102064422, max_depth=9, n_estimators=504,
              subsample=0.7749632785399652)

### 3-3. 결과 파일 생성

In [23]:
pred = lgbm_tune.predict(test_one)
sub = pd.read_csv('sample_submission.csv')
sub['SalePrice'] = pred
sub.to_csv('sub_lgbm_4.csv',index = False)

## 최종 Score: 0.13204